# Reconstruction using inverse problem with measured patterns

In [1]:
import sys
import numpy as np
import scipy

import matplotlib.pyplot as plt

#sys.path.append('//home/crombez/Documents/PhD/python/openspyrit/spyrit/spyrit/learning/') 
from spyrit.misc.statistics import stat_mean_coef_from_model
from spyrit.misc.metrics import *
from spyrit.learning.nets import *
from spyrit.learning.model_Had_1D_DCAN import compNet_1D, Pinv_reg, compNet_1D_test_product, compNet_1D_size_im, compNet_1D_size_im_f,compNet_1D_size_stat,compNet_1D_size_stat_norma

sys.path.append('/home/crombez/Documents/PhD/python/MOPDC/fonction/') 
from data_visualisation import *
from load_data import *
from matrix_tools import *
from walsh_hadamard import *
from walsh_ordered_recon import *

In [2]:
%matplotlib 
#inline 

Using matplotlib backend: Qt5Agg


## Reconstruction parameters

In [3]:
Nl = 256#
Nc = 256#
Nlamb = 128
Nh = 64#

## Hadamard

### Load patterns

In [4]:
save_path = '/home/crombez/Documents/PhD/python/PTND/Data/eGFP_DsRed_can_vs_had/Reconstruction/Mat_rc/'
name = 'motifs_Hadamard_'+str(Nh)+'_'+str(Nc)+'.npy'
motifs = np.load(save_path+name)
plot_im2D(motifs)

### Reconstruction

In [5]:
Pinv = np.linalg.pinv(np.transpose(motifs/np.max(motifs)))

In [8]:
load_path = '/home/crombez/Documents/PhD/python/PTND/Data/eGFP_DsRed_can_vs_had/Preprocess/'
list_run = ['RUN0001','RUN0005']


for run in list_run:

    name = run+'_Had_'+str(Nl)+'_'+str(Nh)+'_'+str(Nlamb)+'.npy'
    hyper_cube = np.load(load_path+name)
    rc_pinv = np.zeros((Nl,Nc,Nlamb))
    
    for i in range(Nlamb):
        rc_pinv[:,:,i] = np.dot(hyper_cube[:,:,i],Pinv)
        
    Save_path = '/home/crombez/Documents/PhD/python/PTND/Data/eGFP_DsRed_can_vs_had/Reconstruction/hyper_cube/'
    file_name = run+'_rc_pinv_'+str(Nl)+'x'+str(Nc)+'x'+str(Nlamb)+'.npy'
    np.save(Save_path+file_name,rc_pinv)

In [10]:
stack = np.zeros((Nl,Nc,))
for i in range(Nlamb):
    stack += rc_pinv[:,:,i]
plot_im2D(stack)

## Cannonique

### Load patterns

In [11]:
Matrc = np.zeros((Nh,Nc))
M = Nc//Nh
j = 0
for i in range(Nh):
    for k in range(M):
        Matrc[i,j] = 1
        j += 1
plot_im2D(Matrc)

In [12]:
Pinv = Matrc

In [13]:
load_path = '/home/crombez/Documents/PhD/python/PTND/Data/eGFP_DsRed_can_vs_had/Preprocess/'
list_run = ['RUN0002','RUN0004']

for run in list_run:

    name = run+'_Can_'+str(Nl)+'_'+str(Nh)+'_'+str(Nlamb)+'.npy'
    hyper_cube = np.load(load_path+name)
    rc_pinv = np.zeros((Nl,Nc,Nlamb))
    
    for i in range(Nlamb):
        rc_pinv[:,:,i] = np.dot(hyper_cube[:,:,i],Pinv)/M
        
    Save_path = '/home/crombez/Documents/PhD/python/PTND/Data/eGFP_DsRed_can_vs_had/Reconstruction/hyper_cube/'
    file_name = run+'_rc_pinv_'+str(Nl)+'x'+str(Nc)+'x'+str(Nlamb)+'.npy'
    np.save(Save_path+file_name,rc_pinv)

## Save lambda stack

In [17]:

Save_path = '/home/crombez/Documents/PhD/python/PTND/Data/eGFP_DsRed_can_vs_had/Reconstruction/hyper_cube/'
stack = np.zeros((Nl,Nc))
run = 'RUN0002'
rc_pinv = np.load(Save_path+run+'_rc_pinv_'+str(Nl)+'x'+str(Nc)+'x'+str(Nlamb)+'.npy')
for i in range(Nlamb):
    stack += rc_pinv[:,:,i]
plot_im2D(stack)

In [18]:

save_path_stack = '/home/crombez/Documents/PhD/python/PTND/Data/eGFP_DsRed_can_vs_had/Analyse_out/stack/'

if not os.path.exists(save_path_stack):
    os.makedirs(save_path_stack)
file_name = run+'_rc_pinv_lambda_stack_'+str(Nl)+'x'+str(Nc)+'.npy'
np.save(save_path_stack+file_name,stack)